In [1]:
import torch
from diffusers import StableDiffusion3Pipeline

# pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-large-turbo", torch_dtype=torch.bfloat16)
# pipe = pipe.to("cuda") 
import sys
sys.path.append("Normalized-Attention-Guidance")

import torch
from processor import JointAttnProcessor2_0
from src.pipeline_sd3_nag import NAGStableDiffusion3Pipeline

model_id = "stabilityai/stable-diffusion-3.5-large-turbo"
if "pipe" not in locals():
  pipe = NAGStableDiffusion3Pipeline.from_pretrained(
      model_id,
      torch_dtype=torch.bfloat16,
  )
pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
import ImageReward as reward
reward_model = reward.load("ImageReward-v1.0")

load checkpoint from /home/wg25r/.cache/ImageReward/ImageReward.pt
checkpoint loaded


In [3]:
import random
seed = int(round(random.random() * 1000000))

In [4]:
import pandas as pd

In [5]:
import wandb
wandb.init(project="VSF")

wandb: Currently logged in as: wguo6358 (3dsmile) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
from ours import inference
import numpy as np
from PIL import Image
import json

with open("prompts.json", "r") as f:
    prompts_data = json.load(f)
    
prompts = pd.read_csv("sampled.csv")
from src.attention_joint_nag import NAGJointAttnProcessor2_0
score_ours = []
score_nag = []

# for i in prompts["prompt"]: 
for i in prompts_data:
    prompt = i["pos"]
    neg_prompt = i["neg"]
    # neg_prompt = "low quality, blurry, bad lighting, poor detail"
    image_ours = inference(pipe, prompt, neg_prompt, seed=seed, scale=2)
    for block in pipe.transformer.transformer_blocks:
        block.attn.processor = NAGJointAttnProcessor2_0()
        
    images = []
    image_nag = pipe(
        prompt,
        nag_negative_prompt=neg_prompt,
        generator=torch.manual_seed(seed),
        guidance_scale=0.,
        nag_scale=4,
        num_inference_steps=8,
        nag_alpha=0.25,
        nag_tau=2.5
    ).images[0]
    rank, scores = reward_model.inference_rank(prompt, [image_ours, image_nag])
    score_ours.append(scores[0])
    score_nag.append(scores[1])
    print(f"Score Ours: {np.mean(score_ours)}, Score NAG: {np.mean(score_nag)}")
    
    wandb.log({
        "img": wandb.Image(Image.fromarray(np.concatenate([np.array(image_ours), np.array(image_nag)], axis=1)), caption=f"+: {prompt}\n -: {neg_prompt}"),
    })   

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Score Ours: 1.2124481201171875, Score NAG: 1.5783334970474243


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Score Ours: 0.4317006667455037, Score NAG: 0.6791097819805145


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 